In [ ]:
import csv

with open('input_file_wenhu.csv', mode='w') as fw:
    with open('input_file_shiyang.csv', mode='w') as fs:
        with open('input_file_yunkai.csv', mode='w') as fy:
            
            #output_wenhu = output[:5000]
            #fileds = ['url1', 'q1', 'a1', 'url2', 'q2', 'a2', 'url3', 'q3', 'a3', 'url4', 'q4', 'a4', 'url5', 'q5', 'a5']
            #writer = csv.DictWriter(fw, fieldnames=fileds)
            #writer.writeheader()
            #for i in range(0, len(output_wenhu) - 5, 5):
            #    writer.writerow({'url1':output_wenhu[i]['url'], 'q1':output_wenhu[i]['q'], 'a1':output_wenhu[i]['a'],
            #                    'url2':output_wenhu[i+1]['url'], 'q2':output_wenhu[i+1]['q'], 'a2':output_wenhu[i+1]['a'],
            #                    'url3':output_wenhu[i+2]['url'], 'q3':output_wenhu[i+2]['q'], 'a3':output_wenhu[i+2]['a'],
            #                    'url4':output_wenhu[i+3]['url'], 'q4':output_wenhu[i+3]['q'], 'a4':output_wenhu[i+3]['a'],
            #                    'url5':output_wenhu[i+4]['url'], 'q5':output_wenhu[i+4]['q'], 'a5':output_wenhu[i+4]['a']})
            
            num = 10
            
            output_shiyang = output[5000:20000]
            fields = []
            for i in range(1, num+1):
                fields.extend(['url{}'.format(i), 'q{}'.format(i), 'a{}'.format(i)])
            writer = csv.DictWriter(fs, fieldnames=fields)
            writer.writeheader()
            for i in range(0, len(output_shiyang) - num, num):
                elem = {}
                for j in range(1, num+1):
                    elem['url{}'.format(j)] = output_shiyang[i+j-1]['url']
                    elem['q{}'.format(j)] = output_shiyang[i+j-1]['q']
                    elem['a{}'.format(j)] = output_shiyang[i+j-1]['a']
                
                writer.writerow(elem) 
            
            
            output_yunkai = output[20000:]
            fields = []
            for i in range(1, num+1):
                fields.extend(['url{}'.format(i), 'q{}'.format(i), 'a{}'.format(i)])
            writer = csv.DictWriter(fy, fieldnames=fields)
            writer.writeheader()
            for i in range(0, len(output_yunkai) - num, num):
                elem = {}
                for j in range(1, num+1):
                    elem['url{}'.format(j)] = output_yunkai[i+j-1]['url']
                    elem['q{}'.format(j)] = output_yunkai[i+j-1]['q']
                    elem['a{}'.format(j)] = output_yunkai[i+j-1]['a']
                
                writer.writerow(elem) 
                

In [ ]:
def regex(inp):
    inp = re.sub(r"> +", r">", inp)
    inp = re.sub(r" +<", r"<", inp)
    inp = re.sub(r">@", r">", inp)
    inp = re.sub(r"\*", r"", inp)
    inp = re.sub(r"#", r"", inp)
    inp = re.sub(r"‡", r"", inp)
    #inp = re.sub(r".0([^0-9])", r"\1", inp)
    for month, abb in [("January", "Jan"), ("February", "Feb"), ("March", "March"), ("April", "April"), 
                   ("May", "May"), ("June","June") , ("July", "July"), ("August", "Aug"), 
                   ("September", "Sep"), ("October", "Oct"), ("November", "Nov"), ("December", "Dec")]:
        inp = re.sub(r"({})([0-9]+)".format(month), r"\1 \2", inp)
        inp = re.sub(r"({})([0-9]+)".format(abb), r"\1 \2", inp)
    return inp

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import jsonlines
import csv
import re
"""
with open('all_html_original.csv', 'w') as fw:
    fields = ['url', 'content']
    writer = csv.DictWriter(fw, fieldnames=fields)
    writer.writeheader()
"""
with open('WikiTableQuestions/wikidata/train.tables.jsonl') as f:
    data = jsonlines.Reader(f)
    for d in data:
        f = '<table class="wikitable"><tr>'
        for h in d['header']:
            f += "<th> {} </th>".format(h)
        f += "</tr>"
        for r in d['rows']:
            f += "<tr>"
            for elem in r:
                f += "<td> {} </td>".format(elem)
            f += "</tr>"
        f += "</table>"
        #writer.writerow({'url': d['id'], "content": f})
        with open('WikiTableQuestions/wikidata/all_html/{}.html'.format(d['id']), 'w') as fw:
            print >> fw, regex(f)

In [ ]:
import re

def regex_equation(line):
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=", r"\1\2+\3+\4+\5=", line)
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)=", r"\1\2+\3+\4=", line)
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)=", r"\1\2+\3=", line)
    #line = re.sub(r"([^+-])([0-9]+)-([0-9]+)+([0-9]+)=", r"\1\2+\3+\4=", line)
    #line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=", r"\1+\2+\3+\4=", line)    
    return line

In [ ]:
with open('/tmp/list.txt') as f:
    for line in f:
        with open('WikiTableQuestions/wikidata/all_html/' + line.strip(), 'r') as f1:
            content = f1.readline().strip()
        with open('WikiTableQuestions/wikidata/all_html/' + line.strip(), 'w') as f1:
            print >> f1, regex_equation(content)
        #print regex_equation(content)

In [ ]:
import json

with open('WikiTableQuestions/wikidata/train_gold.json') as f:
    raw_output = json.load(f)
    output = ["/".join([str(__) for __ in _]) for _ in raw_output]

with open('WikiTableQuestions/wikidata/all_training.tsv', 'w') as fw:
    print >> fw, "id\tutterance\tcontext\ttargetValue"
    with open('WikiTableQuestions/wikidata/train.jsonl') as f:
        data = jsonlines.Reader(f)
        idx = 1
        for d, o in zip(data, output):
            print >> fw, "{}\t{}\t{}\t{}".format(idx, d['question'].replace('\t', ''), d['table_id'], o)
            idx += 1

In [ ]:
import os
import pandas

files = pandas.read_table('WikiTableQuestions/wikidata/all_training.tsv', delimiter="\t")

output = []
for f in os.listdir('WikiTableQuestions/wikidata/all_html/'):
    if "html" in f:
        numbering = f.split('.')[0]
        results = files[files.context == numbering]
        
        for q, a in zip(results.utterance, results.targetValue):
            tmp = {}
            tmp["url"] = 'https://raw.githubusercontent.com/wenhuchen/Interface/master/WikiTableQuestions/wikidata/all_html/{}.html'.format(numbering)
            tmp["q"] = q
            tmp["a"] = a
            output.append(tmp)

In [ ]:
with open('WikiTableQuestions/data/all_training_new.tsv', 'w') as fw:
    with open('WikiTableQuestions/data/all_training.tsv') as f:
        for line in f:
            if "csv" in line:
                try:
                    t1, t2, t3, t4 = line.strip().split('\t')
                except Exception:
                    print line.strip()
                    continue
                    
                try:
                    f1, f2, f3 = t3.split('/')
                except Exception:
                    print t3.strip()
                    continue
                    
                t3 = f2.split('-')[0] + "-" + f3
                new_line = "\t".join([t1, t2, t3, t4])
                print >> fw, new_line
            else:
                print >> fw, line.strip()

In [ ]:
import json
import csv
import pandas

result = pandas.read_csv('results_v2.csv')
filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower():
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

with open('v2_results.json', 'w') as f:
    json.dump(items, f, indent=2)
    
with open('v2_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = items[i + j][0]
            elem["s{}".format(j + 1)] = items[i + j][3]
            
        writer.writerow(elem)

In [ ]:
import json
import pandas

with open('v2_results.json', 'r') as f:
    data = json.load(f)

keys = [_[1] for _ in data]
result = pandas.read_csv('results_v2_new.csv')
filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
        
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower() and r['Input.q{}'.format(i)] not in keys:
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

In [ ]:
import csv

result = pandas.read_csv('results_v3.csv')

filtered = result[result.AssignmentStatus=="Approved"]

new_items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num + 1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in str(r['Answer.d{}'.format(i)]).lower():
            new_items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

new_items = new_items + items

with open('v3_results.json', 'w') as f:
    json.dump(new_items, f, indent=2)
    
num = 5
with open('v3_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(new_items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = new_items[i + j][0]
            elem["s{}".format(j + 1)] = new_items[i + j][3]
            
        writer.writerow(elem)

In [ ]:
s = ""
for month, abb in [("January", "Jan"), ("February", "Feb"), ("March", "March"), ("April", "April"), 
                   ("May", "May"), ("June","June") , ("July", "July"), ("August", "Aug"), 
                   ("September", "Sep"), ("October", "Oct"), ("November", "Nov"), ("December", "Dec")]:
    s += ";s/({})([1-9]+)/\\1 \\2/g;s/({}),([1-9]+)//g".format(month, abb)
print s

In [ ]:
import pandas
import simplejson

result = pandas.read_csv('v2_refine.csv')
filtered_result = result[result.AssignmentStatus=="Approved"]

items = []
num = 10
for i, r in filtered_result.iterrows():
    for i in range(1, num+1):
        items.append((r["Input.url{}".format(i)],"-" ,"-" , r["Answer.d{}".format(i)]))

with open('v2_rewrite.json', 'w') as f:    
    simplejson.dump(items, f, encoding='utf-8', ignore_nan=True)

In [ ]:
import os
#import pandas
import re
import csv

def dealwithNum(inp):
    inp = re.sub(r"([^0-9.])(0)([0-9])%", r"\1\2.\3%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])%", r"\1\2\3.\4%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])%", r"\1\2.\3\4%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])([0-9])%", r"\1\2\3.\4\5%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])([0-9])%", r"\1\2.\3\4\5%", inp)
    
    inp = re.sub(r"10000%", r"100.00%", inp)
    inp = re.sub(r"1000%", r"100.0%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])([0-9])([0-9])%", r"\1\2\3.\4\5\6%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])([0-9])([0-9])%", r"\1\2.\3\4\5\6%", inp)
    
    inp = re.sub(r",([0-9])%", r".\1%", inp)
    inp = re.sub(r",([0-9])([0-9])%", r".\1\2%", inp)
    inp = re.sub(r",([0-9])([0-9])([0-9])%", r".\1\2\3%", inp)
    
    
    #inp = re.sub(r"([0-9]),([0-9])", r"\1\2", inp)
    return inp


with open('all_html.csv', 'w') as f:
    fields = ["url", "content"]
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for f in os.listdir('WikiTableQuestions/wikidata/all_html/'):
        with open('WikiTableQuestions/wikidata/all_html/' + f, 'r') as fr:
            string = fr.readline().strip()
        
        writer.writerow({"url": f, "content": dealwithNum(string)})

#inp_s = "<td>123% <td> 0123% <td>012% <td>1233%  <td>12333% <td>10000% <td>01333%"
#dealwithNum(inp_s)

In [ ]:
import pandas

table = pandas.read_csv('all_html.csv')

for i, item in table.iterrows():
    name = item['url']
    content = item['content']
    with open('WikiTableQuestions/wikidata/all_html/{}'.format(name), 'w') as f:
        print >> f, content

In [ ]:
import json
import csv
import pandas

result = pandas.read_csv('results_v4_1.csv')

filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower():
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

with open('v4_1_results.json', 'w') as f:
    json.dump(items, f, indent=2)

num = 5
with open('v4_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
    
    #fields.extend(["url11", "s11"])
    
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = items[i + j][0]
            elem["s{}".format(j + 1)] = items[i + j][3]
        
        #elem['url11'] = "https://raw.githubusercontent.com/wenhuchen/Interface/master/WikiTableQuestions/wikidata/all_html/2-1236260-1.html"
        #elem['s11'] = "Total is the rank of total"
            
        writer.writerow(elem)

In [3]:
import json
import csv
import pandas

result1 = pandas.read_csv('data/blind1.csv')
result2 = pandas.read_csv('data/blind2.csv')
result = pandas.concat([result1, result2])

filtered = result[result.AssignmentStatus=="Approved"]

num = 5
with open('blind_rewrite_input.csv', 'w') as f:
    fields = []
    fields.append("url1")
    fields.append("wiki1")
    fields.append("topic1")
    fields.extend(['s1', 's2', 's3', 's4', 's5'])
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(filtered)):
        elem = {'url1': filtered.iloc[i]['Input.url1'],
                'wiki1': filtered.iloc[i]['Input.wiki1'],
                'topic1': filtered.iloc[i]['Input.topic1']}
        
        for j in range(num):
            elem["s{}".format(j + 1)] = filtered.iloc[i]['Answer.d{}'.format(j + 1)].lower()
            
        writer.writerow(elem)

In [ ]:
import pandas
import csv
import json
import simplejson

r1 = pandas.read_csv('partial_refine_v2.csv')
r2 = pandas.read_csv('partial_refine_v3.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

results = {}
finished = []
for i, r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}
            results[html_name]['text'].append(t)
            results[html_name]['label'].append(1)
        finished.append(r['Input.s{}'.format(j)])
        
for i, r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}        
            results[html_name]['text'].append(r['Answer.d{}'.format(j)])
            results[html_name]['label'].append(1)
        finished.append(r['Input.s{}'.format(j)])

r1 = pandas.read_csv('partial_neg_v2.csv')
r2 = pandas.read_csv('partial_neg_v3.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

for i, r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            html_name = r['Input.url{}'.format(j)].split('/')[-1]
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}
            if r['Answer.d{}'.format(j)] not in results[html_name]['text']:
                results[html_name]['text'].append(r['Answer.d{}'.format(j)])
                results[html_name]['label'].append(-1)

for i, r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            html_name = r['Input.url{}'.format(j)].split('/')[-1]
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}        
            results[html_name]['text'].append(r['Answer.d{}'.format(j)])
            results[html_name]['label'].append(-1)
            
with open('READY/prelim.json', 'w') as f:
    simplejson.dump(results, f, encoding='utf-8', ignore_nan=True)

In [ ]:
# Prepare results
inp_v2 = pandas.read_csv('v2_rewrite_input.csv')
inp_v3 = pandas.read_csv('v3_rewrite_input.csv')

not_finished = []
done = 0
for i, r in inp_v2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r['s{}'.format(j)] not in finished:
            not_finished.append((r['url{}'.format(j)], r['s{}'.format(j)]))
        else:
            done += 1
        
for i, r in inp_v3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r['s{}'.format(j)] not in finished:
            not_finished.append((r['url{}'.format(j)], r['s{}'.format(j)]))
        else:
            done += 1

num = 5
with open('v23_left_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
    
    #fields.extend(["url11", "s11"])
    
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(not_finished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = not_finished[i + j][0]
            elem["s{}".format(j + 1)] = not_finished[i + j][1]
        writer.writerow(elem)

In [ ]:
result = pandas.read_csv("input_file_yunkai.csv")
keys = set()
for i, r in result.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        keys.add(r["url{}".format(j)])

In [ ]:
result = pandas.read_csv("v4_rewrite_input.csv")
#keys = set()
for i, r in result.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r["url{}".format(j)] in keys:
            keys.remove(r["url{}".format(j)])
unseen_tables = list(keys)

In [ ]:
with open('READY/prelim.json') as f:
    data = json.load(f)

pos = []
neg = []
pos_length = 0
neg_length = 0
for k in data:
    text = data[k]['text']
    label = data[k]['label']
    for t, l in zip(text, label):
        if l == 1:
            pos.append(t)
            pos_length += len(t.split())
        else:
            neg.append(t)
            neg_length += len(t.split())

print len(pos), len(neg)
print (pos_length + 0.0) / len(pos)
print (neg_length + 0.0) / len(neg)

In [ ]:
from collections import Counter

a = Counter()
a.update([1,2,3])
a.update([1,3,4])

print a

In [ ]:
import pandas
import csv

r1 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3580679_batch_results.csv')
r2 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3584639_batch_results.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

r3 = pandas.concat([r1, r2])
finished = []

r1_inp = pandas.read_csv('v23_left_rewrite_input.csv')
r2_inp = pandas.read_csv('v4_rewrite_input.csv')
r3_inp = pandas.concat([r1_inp, r2_inp])

for i, r in r3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        finished.append(r["Input.s{}".format(j)])

print len(finished)
unfinished = []
done = 0
finished_old = []
for i,r in r3_inp.iterrows():
    if "url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        if r["s{}".format(j)]  not in finished:
            unfinished.append((r["url{}".format(j)], r["s{}".format(j)]))
        else:
            finished_old.append(r["s{}".format(j)])
            done += 1

print done
with open('v234_rest_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    num = 5
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(unfinished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = unfinished[i + j][0]
            elem["s{}".format(j + 1)] = unfinished[i + j][1]
            
        writer.writerow(elem)

In [ ]:
r1 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3583922_batch_results.csv')
r2 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3579309_batch_results.csv')
r3 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3579785_batch_results.csv')

r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]
r3 = r3[r3.AssignmentStatus=="Approved"]

r4 = pandas.concat([r1, r2, r3])
finished = []


for i, r in r4.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        finished.append(r["Input.s{}".format(j)])
print len(finished)

r1_inp = pandas.read_csv('v2_rewrite_input.csv')
r2_inp = pandas.read_csv('v3_rewrite_input.csv')
r3_inp = pandas.read_csv('v4_rewrite_input.csv')
r4_inp = pandas.concat([r1_inp, r2_inp, r3_inp])
unfinished = []
done = 0
for i,r in r4_inp.iterrows():
    if "url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        if r["s{}".format(j)]  not in finished:
            unfinished.append((r["url{}".format(j)], r["s{}".format(j)]))
        else:
            done += 1

print done
with open('v234_rest_rewrite_fake_input.csv', 'w') as f:
    num = 5
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(unfinished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = unfinished[i + j][0]
            elem["s{}".format(j + 1)] = unfinished[i + j][1]
            
        writer.writerow(elem)

In [ ]:
import re

string = "faf*\\"
string = string.replace(r'*', '')
string.replace('\\', '')

In [1]:
import re

def substitute(string):
    string = string.lower()
    
    string = string.replace(r"\n", "")
    string = string.replace(r"\\", "")
    string = string.replace(r"!$", "")
    string = string.replace(r"#", "")
    string = string.replace('–', '-')
    string = string.replace('−', '-')
    string = string.replace('−', '-')
    string = string.replace('â', '')
    string = string.replace('“', '')
    string = string.replace('€', '')    
    string = string.replace("√", "")
    string = string.replace(r'"', '')
    string = string.replace(r'\.0 ', '')
    string = string.replace(r'¹', '')
    string = string.replace(r'‡', '')
    string = string.replace(r'«', '')
    string = string.replace(r'»', '')
    string = string.replace(r';', ',')
    string = string.replace(r'$', '')
    string = string.replace(r'£', '')
    string = string.replace(r'¥', '')
    string = string.replace(r'*', '')
    string.replace('\\', '')
    
    string = string.replace(r"'s", " 's")
    string = string.replace(r"'re", " 're")
    string = string.replace(r"'ve", " 've")
    #string = string.replace(r"'nt", " 'nt")
    #string = string.replace(r"'ll", " 'll")
    #string = string.replace(r"\b' ", " 'nt")
    #string = string.replace(r" ' ", "'")
    string = string.replace(r'@', ' ')
    string = string.replace(r'`', '')
    string = string.replace(r'•', ' ')
    string = string.replace(r'·', ' ')
    string = string.replace(r'²', ' square')
    string = string.replace(r'\[', '')
    string = string.replace(r'\]', '')
    string = string.replace(r'\{', '')
    string = string.replace(r'\}', '')
    string = string.replace(r'\|', '')
    string = string.replace(r'\^', '')
    string = string.replace(r'”', '')

    string = re.sub(r'\[.+\]', ' ', string)
    string = re.sub(r',([0-9]{3})(?![0-9])', r'\1', string)

    string = re.sub(r'([^,]),', r"\1 ,", string)
    string = re.sub(r',([^,])', r", \1", string)
    string = re.sub(r'([^(])\(', r"\1 (", string)
    string = re.sub(r'\)([^)])', r") \1", string)
    string = re.sub(r'\.\)', r")", string)
    string = re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=', r"\1+\2+\3+\4=", string)
    string = re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)=', r"\1+\2+\3=", string)
    string = re.sub(r'([0-9]+)-([0-9]+)=', r"\1+\2=", string)
    string = re.sub(r"' +([0-9])", r"'\1", string)

    string = re.sub(r"([^ ])\+", r"\1 +", string)
    string = re.sub(r"\+([^ ])", r"+ \1", string)
    string = re.sub(r"([^ ])=", r"\1 =", string)
    string = re.sub(r"=([^ ])", r"= \1", string)

    string = re.sub(r"-([^- ])", r"- \1", string)
    string = re.sub(r"([^- ])-", r"\1 -", string)
    string = re.sub(r"-([^- ])", r"- \1", string)
    string = re.sub(r"/([^ ])", r"/ \1", string)
    string = re.sub(r"([^ ])/", r"\1 /", string)
    string = string.replace(r'()', '')
    string = re.sub(r'([^0-9])\.', r'\1', string)

    string = re.sub(r'([^0-9 ]):', r"\1 :", string)
    string = re.sub(r':([^0-9 ])', r": \1", string)
    string = re.sub(r'([0-9])pm', r"\1 pm", string)
    string = re.sub(r'([0-9])am', r"\1 am", string)
    string = re.sub(r'([0-9])rpm', r"\1 rpm", string)
    string = re.sub(r'([0-9])km', r"\1 km", string)
    string = re.sub(r'([0-9])cm', r"\1 cm", string)
    string = re.sub(r'([0-9])m', r"\1 m", string)
    string = re.sub(r'([0-9])mm', r"\1 mm", string)
    string = re.sub(r'([0-9])kg', r"\1 kg", string)
    string = re.sub(r'([0-9])g', r"\1 g", string)
    string = re.sub(r'([0-9])kw', r"\1 kw", string)
    string = re.sub(r'([0-9])kv', r"\1 kv", string)
    string = re.sub(r'([0-9])mph', r"\1 mph", string)
    #string = re.sub(r'([0-9])@', r"\1 @", string)
    #string = re.sub(r'@([0-9])', r"@ \1", string)
    string = re.sub(r'category : articles with hcards', r"", string)
    string = re.sub(r'category : articles with hcard', r"", string)
    string = re.sub(r'category : articles without hcard', r"", string)
    
    string = re.sub(r' 1 time', r" one time", string)
    string = re.sub(r' 2 time', r" two time", string)
    string = re.sub(r' 3 time', r" three time", string)
    string = re.sub(r' 4 time', r" four time", string)
    string = re.sub(r' 5 time', r" five time", string)
    string = re.sub(r' 6 time', r" six time", string)
    string = re.sub(r' 7 time', r" seven time", string)
    string = re.sub(r' 8 time', r" eight time", string)
    string = re.sub(r' 9 time', r" nine time", string)
    string = re.sub(r' 10 time', r" ten time", string)
    string = re.sub(r' 11 time', r" eleven time", string)
    string = re.sub(r' 12 time', r" twelve time", string)
    string = re.sub(r' 13 time', r" thirteen time", string)
    string = re.sub(r' 14 time', r" fourteen time", string)
    string = re.sub(r' 15 time', r" fifteen time", string)
    string = re.sub(r' 16 time', r" sixteen time", string)
    string = re.sub(r' 17 time', r" seventeen time", string)
    string = re.sub(r' 18 time', r" eighteen time", string)
    string = re.sub(r' 19 time', r" nineteen time", string)
    string = re.sub(r' 20 time', r" twenty time", string)
    
    string = re.sub(r'once', r"one time", string)
    string = re.sub(r'twice', r"two times", string)  
    string = re.sub(r"\.+$", '', string)
    string = re.sub(r',+$', '', string)    
    string = re.sub(r'\s+', ' ', string)
    string = re.sub(r'\.+','.',string)
    string = re.sub(r',+',',',string)
    string = re.sub(r'^ ', '', string)
    string = re.sub(r' $', '', string)
    #string = re.sub('70 - 76 - 68 - 214', '70 + 76 + 68 = 214', string)
    string = re.sub('turkish cup was 56', 'turkish cup was 54', string)
    #string = re.sub('brigido iriarte', 'brígido iriarte', string)
    return string

#print substitute("fa\g##ga,/// she got(ff)ff...")
string = "he hasn't done it"
print re.sub(r"n't", " n't", string)

he has n't done it


In [5]:
#substitute('the new york jets won in san diego againtst the chargers by 7 points (34-27) in september of 1969.')
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import json
with open('challenge/blind_test.json') as f:
    data = json.load(f)
new_data = {}
for k, d in data.items():
    new_data[k] = [substitute(d[0]), d[1], d[2]]

In [7]:
with open('challenge/blind_test_tokenized.json', 'w') as f:
    json.dump(new_data, f, indent=2)

In [11]:
def blacklist(string):
    black = ['bye', "no chart for", "km (mi)", "lb·ft", "ft (m)", "kg (lb)", 
             "a report of report", "report was report", "is 'report'", "?"]
    for b in black:
        if b in string:
            return True
    return False

with open('data/short_subset.txt') as f:
    limit_length = [_.strip() for _ in f.readlines()]

In [ ]:
# encoding=utf8
import sys
import pandas

r1_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_0.csv")
r1_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_1.csv")

r1 = pandas.concat([r1_1, r1_2])
r1 = r1[r1.AssignmentStatus=="Approved"]

f = open('/tmp/output.txt', 'w')

index = 0
finished = {}
trash = []
for i,r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1] + '.csv'
        if html_name not in limit_length:
            continue
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            #trash.append(html_name)
            print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [1]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(1)
                    
r2_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_2.csv")
r2_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_3.csv")
r2_3 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_4.csv")

r2 = pandas.concat([r2_1, r2_2, r2_3])
r2 = r2[r2.AssignmentStatus=="Approved"]

for i,r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1] + '.csv'
        if html_name not in limit_length:
            continue     
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [1]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(1)


r3_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_0.csv")
r3_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_1.csv")
r3_3 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_2.csv")
r3_4 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_3.csv")

r3 = pandas.concat([r3_1, r3_2, r3_3, r3_4])
r3 = r3[r3.AssignmentStatus=="Approved"]

f = open('/tmp/output.txt', 'w')

index = 0
for i,r in r3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1] + '.csv'
        if html_name not in limit_length:
            continue
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [0]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(0)

f.close()
print index
import json
with open("READY/pre_training_all.json", 'w') as f:
    json.dump(finished, f, indent=2)

In [ ]:
print trash
import os
import shutil

print trash
for l in trash + ["2-11916083-12.html", "2-1236260-1.html"]:
    try:
        shutil.move("data/all_csv/" + l + ".csv", "data/trash_csv/" + l + ".csv")
    except Exception:
        print "error, {}".format("data/all_csv/" + l + ".csv", "data/trash_csv/" + l + ".csv")

In [ ]:
from bs4 import BeautifulSoup
import urllib2
import csv

url = open('WikiTableQuestions/wikidata/all_html/2-18178551-5.html').readline().strip()
soup = BeautifulSoup(url, "html.parser")

table = soup.findAll("table", {"class":"wikitable"})[0]
rows = table.findAll("tr")

with open('/tmp/editors.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        csv_row = []
        for cell in row.findAll(["td", "th"]):
            csv_row.append(cell.get_text())
        writer.writerow(csv_row)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import csv
import os
from bs4 import BeautifulSoup

for filename in os.listdir('WikiTableQuestions/wikidata/all_html/'):
    if filename.endswith(".html"):
        url = open('WikiTableQuestions/wikidata/all_html/' + filename).readline().strip()
        soup = BeautifulSoup(url, "html.parser")
        table = soup.findAll("table", {"class":"wikitable"})
        if len(table) > 0:
            table = table[0]
            rows = table.findAll("tr")
            with open('WikiTableQuestions/wikidata/all_csv/' + filename + '.csv', 'w') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter='#')
                for row in rows:
                    csv_row = []
                    for cell in row.findAll(["td", "th"]):
                        print cell.get_text()
                        csv_row.append(substitute(cell.get_text()))
                        print substitute(cell.get_text())
                        print ""
                    spamwriter.writerow(csv_row)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv
import urllib
import json
import os

with open('READY/r1_training_all.json') as f:
    data = json.load(f)
    
files = data.keys()

unseen = []
with open('data/short_subset.txt') as fs:
    for f in fs:
        f = f.strip()
        if f not in files and f in tiny_mapping:
            unseen.append(f)
with open("data/v5_unseen.json", 'w') as f:
    json.dump(unseen, f)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import urllib
import json
import os 
import csv

with open('data/table_to_page_new.json', 'r') as f:
    tiny_mapping = json.load(f)
    
with open("data/v5_unseen.json", 'r') as f:
    unseen = json.load(f)
    
with open('v5_write_input.csv', 'w') as f:
    fields = ["url1", "wiki1", "topic1"]
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()

    for k in unseen:
        if k in tiny_mapping:
            writer.writerow({"url1": 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k, 
                            "wiki1": tiny_mapping[k][1], "topic1": tiny_mapping[k][0]})
        else:
            writer.writerow({"url1": 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k, "wiki1": "javascript:void(0)", "topic1": "None"})

In [ ]:
with open('/tmp/table_to_page.json', 'r') as f:
    old_tiny_mapping1 = json.load(f)
with open('data/table_to_page.json', 'r') as f:
    old_tiny_mapping2 = json.load(f)
    
new_tiny_mapping = {}
for k, v in old_tiny_mapping2.iteritems():
    new_tiny_mapping[k] = [substitute(old_tiny_mapping1[k].split('/')[-1].replace('_', ' ')), old_tiny_mapping2[k]]

with open('data/table_to_page_new.json', 'w') as f:
    json.dump(new_tiny_mapping, f, indent=2)

In [ ]:
with open('/tmp/g.txt', 'w') as f:
    x = "https://en.wikipedia.org/wiki/2007%E2%80%9308_Scottish_Second_Division"
    print >> f, ["topic", urllib.unquote(x).split('/')[-1]]
    x = 'https://en.wikipedia.org/wiki/Ana_Timoti%C4%87'
    print >> f, {"topic": urllib.unquote(x).split('/')[-1]}

In [ ]:
import pandas

name = files[0]
tab = pandas.read_csv('data/all_csv/' + name + '.csv', delimiter='#')
sent = data[name][0][0]
label = data[name][1][0]

tab

In [ ]:
from fuzzywuzzy import fuzz

activations = []
for i, t in enumerate(tab.columns):
    for j, v in enumerate(tab[t]):
        if isinstance(v, str) and len(v.split(' ')) > 2:
            if fuzz.partial_ratio(v, sent) > 95:
                print t, j
        else:
            if str(v) in sent:
                print t, j

In [ ]:
import pandas
from collections import Counter
import pprint

t = pandas.read_csv("/Users/wenhuchen/Downloads/Batch_3598504_batch_results.csv")
t = t[t.AssignmentStatus=="Approved"]


def transform(string):
    if string == "Problematic":
        return "Neutral"
    else:
        return string

kv_pairs = {}
num = 5
for i, r in t.iterrows():
    for j in range(1, num + 1):
        csv_name = r['Input.url{}'.format(j)].split('/')[-1]
        if (csv_name, r['Input.s{}'.format(j)]) not in kv_pairs:
            kv_pairs[(csv_name, r['Input.s{}'.format(j)])] = [r["Input.o{}".format(j)], r["Answer.A{}".format(j)], "None"]
        else:
            kv_pairs[(csv_name, r['Input.s{}'.format(j)])][2] = r["Answer.A{}".format(j)]

#new_kv_pairs = {}
#for k, v in kv_pairs.iteritems():
#    if "Problematic" not in v:
#        new_kv_pairs[k] = v
        #if v[1] == v[2]:
        #    new_kv_pairs[k][0] = v[1]
            
counter = Counter()
p_c = [] 
for k, v in kv_pairs.iteritems():
    tmp = set(v)
    if len(tmp) == 1:
        p_c.append(1)
    elif len(tmp) == 2:
        p_c.append(1/3.)
    else:
        p_c.append(0)
    
    counter.update(v)

p_e = []
for i, j in counter.items():
    p_e.append(j + 0.0)

p_e = [_ / sum(p_e) for _ in p_e]
p_e = sum(_**2 for _ in p_e)

p_c = sum(p_c) / len(p_c)

kappa = (p_c - p_e) / (1 - p_e)
print "kappa = {}".format(kappa)
print counter
for k, v in new_kv_pairs.iteritems():
    if v[0] == 'Contradictory':
        print k
        print v

In [ ]:
cleaned_pairs = {}

def decide(s1, s2, s3):
    s2int = {'Entailed': 1, 'Neutral': 0, 'Contradictory': -1}
    avg_score = (s2int[s1] + s2int[s2] + s2int[s3]) // 3.
    if avg_score > 0:
        return "Entailed"
    elif avg_score < 0:
        return "Contradictory"
    else:
        return "Neutral"

for k, v in new_kv_pairs.iteritems():
    if k[0] not in cleaned_pairs:
        cleaned_pairs[k[0]] = [[k[1]], [decide(*v)]]
    else:
        cleaned_pairs[k[0]][0].append(k[1])
        cleaned_pairs[k[0]][1].append(decide(*v))

In [ ]:
import json
with open('READY/cleaned.json', 'w') as f:
    json.dump(cleaned_pairs, f)

In [ ]:
import pandas

t = pandas.read_csv('/Users/wenhuchen/Downloads/clean/negative_partial.csv')
t = t[t.AssignmentStatus=="Approved"]

print len(t)
finished = []
for i, row in t.iterrows():
    for j in range(1, 11):
        finished.append(row['Input.s{}'.format(j)])

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import csv
import random

with open('READY/r1_training_all.json') as f:
    data = json.load(f)

with open('data/table_to_page.json') as f:
    mapping = json.load(f)

dummy = [
    # refuted
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'chicago bears has won the game in september',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    # erroneous
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'chicago bears is the best team in the league',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'the ninth week against atlant falcons was a big win with 35 - 6',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'the result against st. lous cardinals was a lost',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'san francisco 49ers was competing on noveber 28 , 1982',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
        [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'san francisco is is opponent where week is number',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ]
]
    
num = 10
count = 0
with open('verify_inputs_fake.csv', 'w') as fs:
    fields = []
    for i in range(0, num + 1):
        fields.extend(['wiki{}'.format(i), 'url{}'.format(i), 's{}'.format(i), 'topic{}'.format(i)])
    csvwriter = csv.DictWriter(fs, fieldnames=fields)
    csvwriter.writeheader()

    seed = random.choice(range(0, len(dummy)))
    buf = {'wiki0': dummy[seed][3], 'topic0': dummy[seed][2], 'url0': dummy[seed][0], 's0': dummy[seed][1]}
    for k, v in data.iteritems():
        entry = data[k]
        for sent, lab in zip(entry[0], entry[1]):
            if lab == 0:
                cur = len(buf) // 4
                if 'url10' in buf:
                    csvwriter.writerow(buf)
                    seed = random.choice(range(0, len(dummy)))
                    buf = {'wiki0': dummy[seed][3], 'topic0': dummy[seed][2], 'url0': dummy[seed][0], 's0': dummy[seed][1]}
                    cur = 1
                    count += 1
                if sent in finished:
                    continue
                buf['url{}'.format(cur)] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
                buf['s{}'.format(cur)] = sent
                if k in mapping:
                    buf['wiki{}'.format(cur)] = mapping[k][1]
                    buf['topic{}'.format(cur)] = mapping[k][0]
                else:
                    buf['wiki{}'.format(cur)] = "#"
                    buf['topic{}'.format(cur)] = "None"
        if count % 500 == 0:
            print count
        #if count > 30:
        #    break
            

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import csv

with open('READY/r1_training_all.json') as f:
    data = json.load(f)

num = 11
count = 0
with open('verify_inputs_fake.csv', 'w') as fs:
    fields = []
    for i in range(0, num):
        fields.extend(['wiki{}'.format(i), 'url{}'.format(i), 's{}'.format(i), 'o{}'.format(i)])
    csvwriter = csv.DictWriter(fs, fieldnames=fields)
    csvwriter.writeheader()

    buf = {}
    for k, v in data.iteritems():
        entry = data[k]
        for sent, lab in zip(entry[0], entry[1]):
            cur = len(buf) // 3 + 1
            if cur > num:
                csvwriter.writerow(buf)
                buf = {}
                cur = 1
                count += 1
            buf['url{}'.format(cur)] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            buf['s{}'.format(cur)] = sent
            if lab == 1:
                buf['o{}'.format(cur)] = "Entailed"
            else:
                buf['o{}'.format(cur)] = "Contradictory"

In [ ]:
import json

data = []
with open('all_sources/full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line.strip()))
    #mapping[str(d['goldAnnotation']['titleId']) + '-' + str(d['tableId'])] = d['goldAnnotation']['title']

In [ ]:
mapping = {}
for d in data:
    mapping[str(d['pgId']) + '-' + str(d['tableId'])] = d['pgTitle']

In [ ]:
import os
import json

tiny_mapping = {}
for f in os.listdir('data/all_csv/'):
    if f.endswith('.csv'):
        _, pageid, tableid = f.split('.')[0].split('-')
        if pageid + '-' + tableid in mapping:
            tiny_mapping[f] = "https://en.wikipedia.org/wiki/" + "_".join(mapping[pageid + '-' + tableid].split(' '))

with open('data/table_to_page.json', 'w') as f:
    json.dump(tiny_mapping, f, indent=2)

In [ ]:
import json

with open('data/table_to_page.json', 'r') as f:
    tiny_mapping = json.load(f)

print len(tiny_mapping)
print tiny_mapping['1-18974269-1.html.csv']
#print tiny_mapping['1-1007688-1.html.csv']

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import urllib
import json

with open('data/table_to_page.json', 'r') as f:
    tiny_mapping = json.load(f)
    
new_tiny_mapping = {}
for k, v in tiny_mapping.iteritems():
    new_tiny_mapping[k] = "https://en.wikipedia.org/wiki/" + urllib.quote(v[30:].encode('utf8'))

with open('data/table_to_page_new.json', 'w') as f:
    json.dump(new_tiny_mapping, f, indent=2)

In [ ]:
import pandas

t = pandas.read_csv('v5_write_input.csv')
length = len(t)

t = t.head(3000).tail(2000)
#t = t.head(1000)

t.to_csv('v5_write_input_1000_3000.csv', index=False)

t

In [ ]:
import json
import pandas

t = pandas.read_csv("/Users/wenhuchen/Downloads/harvest-v2/first1000.csv")
print len(t)
t = t[t.AssignmentStatus=="Approved"]
print len(t)
index = 0
num = 5
finished = {}
for i,r in t.iterrows():
    html_name = r['Input.url1']
    html_name = html_name.split('/')[-1]
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        #if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
        #    print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
            replaced_sent = substitute(orig_input)
            #replaced_sent = orig_input
            index += 1
            if html_name not in finished:
                finished[html_name] = [[replaced_sent], [1]]
            else:
                finished[html_name][0].append(replaced_sent)
                finished[html_name][1].append(1)

with open("READY/round2_first1000.json",'w') as f:
    json.dump(finished, f, indent=2)

In [ ]:
import json
import pandas

t = pandas.read_csv("/Users/wenhuchen/Downloads/harvest-v2/last4000.csv")
print len(t)
t = t[t.AssignmentStatus=="Approved"]
print len(t)
index = 0
num = 5
finished = {}
for i,r in t.iterrows():
    html_name = r['Input.url1']
    html_name = html_name.split('/')[-1]
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        #if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
        #    print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
            replaced_sent = substitute(orig_input)
            #replaced_sent = orig_input
            index += 1
            if html_name not in finished:
                finished[html_name] = [[replaced_sent], [1]]
            else:
                finished[html_name][0].append(replaced_sent)
                finished[html_name][1].append(1)

with open("READY/round2_last4000.json",'w') as f:
    json.dump(finished, f, indent=2)

In [ ]:
import json
import pandas

t = pandas.read_csv("/Users/wenhuchen/Downloads/harvest-v2/middle2000.csv")
print len(t)
t = t[t.AssignmentStatus=="Approved"]
print len(t)
index = 0
num = 5
finished = {}
for i,r in t.iterrows():
    html_name = r['Input.url1']
    html_name = html_name.split('/')[-1]
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        #if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
        #    print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
            replaced_sent = substitute(orig_input)
            #replaced_sent = orig_input
            index += 1
            if html_name not in finished:
                finished[html_name] = [[replaced_sent], [1]]
            else:
                finished[html_name][0].append(replaced_sent)
                finished[html_name][1].append(1)

with open("READY/round2_middle2000.json",'w') as f:
    json.dump(finished, f, indent=2)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv

with open("READY/round2_first1000.json") as f:
    data = json.load(f)

with open("data/table_to_page.json") as f:
    mapping = json.load(f)
    
fields = ['url1', 'wiki1', 'topic1', 's1', 's2', 's3', 's4', 's5']
index = 0
with open("rewrite_fake_first1000.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k, v in data.iteritems():
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            index += 1
        #if index > 20:
        #    break

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv

with open("READY/round2_middle2000.json") as f:
    data = json.load(f)

with open("data/table_to_page.json") as f:
    mapping = json.load(f)
    
fields = ['url1', 'wiki1', 'topic1', 's1', 's2', 's3', 's4', 's5']
index = 0
with open("rewrite_fake_middle2000.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k, v in data.iteritems():
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            index += 1
        #if index > 20:
        #    break

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv

with open("READY/round2_last4000.json") as f:
    data = json.load(f)

files = data.keys()
num = 1473
shiyang_data = files[:num]
wanghong_data = files[num:2*num]
yunkai_data = files[2*num:]
    
with open("data/table_to_page.json") as f:
    mapping = json.load(f)
    
fields = ['url1', 'wiki1', 'topic1', 's1', 's2', 's3', 's4', 's5']

with open("rewrite_fake_shiyang.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k in shiyang_data:
        v = data[k]
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            
with open("rewrite_fake_yunkai.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k in yunkai_data:
        v = data[k]
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            
with open("rewrite_fake_wanghong.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k in wanghong_data:
        v = data[k]
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)


In [ ]:
import pandas

t = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v3/partial_positive.csv')
t = t[t.AssignmentStatus=="Approved"]

mapping = {}
for i, r in t.iterrows():
    for i in range(1, 6):
        name = r['Input.url{}'.format(i)].split('/')[-1]
        if name not in mapping:
            mapping[name] = [[r['Input.s{}'.format(i)]], [r['Answer.A{}'.format(i)]]]
        else:
            mapping[name][0].append(r['Input.s{}'.format(i)])
            mapping[name][1].append(r['Answer.A{}'.format(i)])
import json

with open('READY/verify.json', 'w') as f:
    json.dump(mapping, f, indent=2)

In [ ]:
import re
import string
import json
stop_words = ['be', 'she', 'he', 'her', 'his', 'their', 'the', 'it', ',', '.', '-', 'also', 'will', 'would', 'this', 'that',
             'these', 'those', 'well', 'with', 'on', 'at', 'and', 'as', 'for', 'from', 'in', 'its', 'of', 'to', 'a',
             'an', 'where', 'when', 'by', 'not', "'s", "'nt", "make", 'who', 'have', 'within', 'without', 'what',
             'during', 'than', 'then', 'if', 'when', 'while', 'time', 'appear', 'attend', 'every', 'one', 'two', 'over',
             'both', 'above', 'only']

def get_closest(string, indexes, tab):
    dist = 10000
    len_string = len(string)
    for index in indexes:
        len_tab = len(tab[index[0]][index[1]])
        if abs(len_tab - len_string) == 0:
            return index
        elif abs(len_tab - len_string) < dist:
            minimum = index
            dist = abs(len_tab - len_string)
    return minimum

def postprocess(inp, backbone, tabs):
    inp = re.sub(r'([^0-9])\.', r'\1', inp)
    inp = re.sub(r'(\b)one(\b)', r'\g<1>1\g<2>', inp)
    inp = re.sub(r'(\b)is one(\b)', r'\g<1>is 1\g<2>', inp)
    inp = re.sub(r'(\b)two(\b)', '\g<1>2\g<2>', inp)
    inp = re.sub(r'(\b)three(\b)', '\g<1>3\g<2>', inp)
    inp = re.sub(r'(\b)four(\b)', '\g<1>4\g<2>', inp)
    inp = re.sub(r'(\b)five(\b)', '\g<1>5\g<2>', inp)
    inp = re.sub(r'(\b)six(\b)', '\g<1>6\g<2>', inp)
    inp = re.sub(r'(\b)seven(\b)', '\g<1>7\g<2>', inp)
    inp = re.sub(r'(\b)eight(\b)', '\g<1>8\g<2>', inp)
    inp = re.sub(r'(\b)nine(\b)', '\g<1>9\g<2>', inp)
    inp = re.sub(r'(\b)ten(\b)', '\g<1>10\g<2>', inp)
    inp = re.sub(r'(\b)eleven(\b)', '\g<1>11\g<2>', inp)
    inp = re.sub(r'(\b)twelve(\b)', '\g<1>12\g<2>', inp)
    inp = re.sub(r'(\b)thirteen(\b)', '\g<1>13\g<2>', inp)
    inp = re.sub(r'(\b)fourteen(\b)', '\g<1>14\g<2>', inp)
    inp = re.sub(r'(\b)fifteen(\b)', '\g<1>15\g<2>', inp)
    inp = re.sub(r'(\b)sixteen(\b)', '\g<1>16\g<2>', inp)
    inp = re.sub(r'(\b)seventeen(\b)', '\g<1>17\g<2>', inp)
    inp = re.sub(r'(\b)eighteen(\b)', '\g<1>18\g<2>', inp)
    inp = re.sub(r'(\b)nineteen(\b)', '\g<1>19\g<2>', inp)
    inp = re.sub(r'(\b)twenty(\b)', '\g<1>20\g<2>', inp)
    new_str = []
    new_tags = []
    buf = ""
    last = set()
    inp, pos_tags = get_lemmatize(inp, True)
    for w, p in zip(inp, pos_tags):
        if w in backbone:
            if buf == "":
                last = set(backbone[w])
                buf = w
            else:
                proposed = set(backbone[w]) & last
                if not proposed:
                    if buf not in stop_words:
                        closest = get_closest(buf, last, tabs)
                        buf = '#' + buf + '#' + json.dumps(closest)
                    new_str.append(buf)
                    new_tags.append('ENT')
                    buf = w
                    last = set(backbone[w])
                else:
                    buf += " " + w
                    last = proposed
        else:
            if buf != "":
                if buf not in stop_words:
                    closest = get_closest(buf, last, tabs)
                    buf = '#' + buf + '#' + json.dumps(closest)
                new_str.append(buf)
                new_tags.append('ENT')
            buf = ""
            last = set()
            new_str.append(w)
            new_tags.append(p)
    
    if buf != "":
        if buf not in stop_words:
            closest = get_closest(buf, last, tabs)
            buf = '#' + buf + '#' + json.dumps(closest)
        new_str.append(buf)
        new_tags.append("ENT")
    #inp = re.sub(r'ʼ', '', inp)
    #inp = re.sub(r'ʻ', '', inp)
    #inp = re.sub(r' ?- ?', '-', inp)
    #inp = re.sub(r' ?\+ ?', '+', inp)
    #inp = re.sub(r' ?/ ?', '/', inp)
    #inp = re.sub(r" ?'s", "'s", inp)
    #inp = re.sub(r" ?'nt", "'nt", inp)
    #inp = inp
    return " ".join(new_str), " ".join(new_tags)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import pandas
import json

with open('data/short_subset.txt') as f:
    limit_length = [_.strip() for _ in f.readlines()]
    
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

with open('data/table_to_page.json', 'r') as f:
    mapping = json.load(f)
    
t = pandas.read_csv('/Users/wenhuchen/Downloads/harvest/positive.csv')
t = t[t.AssignmentStatus=="Approved"]

print len(t) * 10
num = 10
results = {}
count = 0
for i, row in t.iterrows():
    for j in range(1, num + 1):
        if row['Answer.A{}'.format(j)] == "Entailed":
            name = row['Input.url{}'.format(j)].split('/')[-1]
            if name in limit_length:
                count += 1
                if name in results:
                    sent = substitute(row['Input.s{}'.format(j)])
                    results[name][0].append(sent)
                    results[name][1].append(1)
                else:
                    sent = substitute(row['Input.s{}'.format(j)])
                    results[name] = [[sent], [1], mapping.get(name, ["none", "none"])[0]]

t1 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest/negative_1.csv')
t2 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest/negative_2.csv')
t = pandas.concat([t1, t2])
t = t[t.AssignmentStatus=="Approved"]

print len(t) * 10
for i, row in t.iterrows():
    for j in range(1, num + 1):
        if row['Answer.A{}'.format(j)] == "Refuted":
            name = row['Input.url{}'.format(j)].split('/')[-1]
            if name in limit_length:
                count += 1
                if name in results:
                    sent = substitute(row['Input.s{}'.format(j)])
                    results[name][0].append(sent)
                    results[name][1].append(0)
                else:
                    sent = substitute(row['Input.s{}'.format(j)])
                    results[name] = [[sent], [0], mapping.get(name, ["none", "none"])[0]]
print count

with open('READY/r1_training_all.json', 'w') as f:
    json.dump(results, f, indent=2)

In [ ]:
import os
import pandas

with open('data/short_subset.txt') as f:
    limit_length = [_.strip() for _ in f.readlines()]

for j, f in enumerate(limit_length):
    print "------------", f, "------------"
    t = pandas.read_csv('data/all_csv/' + f, '#')
    for i in t.columns:
        print i, ":", t.at[0, i]
    if j > 100:
        break

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
tag_dict = {"JJ": wordnet.ADJ,
            "NN": wordnet.NOUN,
            "NNS": wordnet.NOUN,
            "NNP": wordnet.NOUN,
            "NNPS": wordnet.NOUN,
            "VB": wordnet.VERB,
            "VBD": wordnet.VERB,
            "VBG": wordnet.VERB,
            "VBN": wordnet.VERB,
            "VBP": wordnet.VERB,
            "VBZ": wordnet.VERB,
            "RB": wordnet.ADV,
            "RP": wordnet.ADV}

lemmatizer = WordNetLemmatizer()

def get_lemmatize(words, return_pos):
    #words = nltk.word_tokenize(words)
    words = words.split(' ')
    pos_tags = [_[1] for _ in nltk.pos_tag(words)]
    word_roots = []
    for w, p in zip(words, pos_tags):
        if is_ascii(w) and p in tag_dict:
            word_roots.append(lemmatizer.lemmatize(w, tag_dict[p]))
        else:
            word_roots.append(w)
    if return_pos:
        return word_roots, pos_tags
    else:
        return word_roots

In [ ]:
import pandas

t1 = pandas.read_csv('/Users/wenhuchen/Downloads/clean/Batch_3625468_batch_results.csv')
t2 = pandas.read_csv('/Users/wenhuchen/Downloads/clean/negative_partial.csv')

t = pandas.concat([t1, t2])

t.to_csv('/Users/wenhuchen/Downloads/clean/negative.csv')

In [ ]:
import pandas

t1 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3618313_batch_results.csv')
t2 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3618582_batch_results.csv')
t3 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3623032_batch_results.csv')
t4 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3618599_batch_results.csv')

t = pandas.concat([t1, t2, t3, t4])
t.to_csv('/Users/wenhuchen/Downloads/harvest-v2/negative.csv')

t1 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/first1000.csv')
t2 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/middle2000.csv')
t3 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/last4000.csv')

t = pandas.concat([t1, t2, t3])
t.to_csv('/Users/wenhuchen/Downloads/harvest-v2/positive.csv')

In [13]:
# encoding=utf8
import sys
import pandas
import json
from collections import Counter

r1_1 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/first1000.csv')
r1_2 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/middle2000.csv')
r1_3 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/last4000.csv')

r1 = pandas.concat([r1_1, r1_2, r1_3])
r1 = r1[r1.AssignmentStatus=="Approved"]

with open('data/table_to_page.json', 'r') as f:
    mapping = json.load(f)
    
index = 0
finished = {}
trash = []
for i,r in r1.iterrows():
    num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url1'].split('/')[-1]
        if html_name in limit_length:
            orig_input = r["Answer.d{}".format(j)]   
            if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
                replaced_sent = substitute(orig_input)
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [1], mapping.get(html_name, ["none", "none"])[0]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(1)
                    
r2_1 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3618313_batch_results.csv')
r2_2 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3618582_batch_results.csv')
r2_3 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3623032_batch_results.csv')
r2_4 = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/Batch_3618599_batch_results.csv')

r2 = pandas.concat([r2_1, r2_2, r2_3, r2_4])
r2 = r2[r2.AssignmentStatus=="Approved"]

for i,r in r2.iterrows():
    num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url1'].split('/')[-1]
        if html_name in limit_length:
            orig_input = r["Answer.d{}".format(j)]   
            if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
                replaced_sent = substitute(orig_input)
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [0], mapping.get(html_name, ["none", "none"])[0]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(0)

ValueError: Extra data: line 1 column 8 - line 595 column 3302 (char 7 - 2142109)

In [17]:
with open("/Users/wenhuchen/Downloads/harvest-v2/agreement.csv") as f:
    data = pandas.read_csv(f)

data = data[data.AssignmentStatus=="Approved"]

succ, fail = 0, 0
fields = 10
sent = {}

tp, tn, fp, fn = 0, 0, 0, 0
orig_corr, orig_fals = 0, 0
pred_corr, pred_fals = 0, 0
for i, row in data.iterrows():
    for j in range(1, fields + 1):
        if row['Input.label{}'.format(j)] == "Entailed" and row['Answer.A{}'.format(j)] == "Entailed":
            tp += 1
        elif row['Input.label{}'.format(j)] == "Entailed" and row['Answer.A{}'.format(j)] == "Refuted":
            fn += 1
        elif row['Input.label{}'.format(j)] == "Refuted" and row['Answer.A{}'.format(j)] == "Entailed":
            fp += 1
        else:
            tn += 1
        
        label = row['Answer.A{}'.format(j)]
        if row['Input.s{}'.format(j)] in sent:
            sent[row['Input.s{}'.format(j)]].append(label)
        else:
            sent[row['Input.s{}'.format(j)]] = [label]

for k, v in finished.iteritems():
    for i in range(len(v[0])):
        if v[0][i] in sent:
            counter = Counter(sent[v[0][i]])
            if counter['Entailed'] > counter['Refuted']:
                v[1][i] = 1
            else:
                v[1][i] = 0

with open("READY/r2_training_all.json", 'w') as f:
    json.dump(finished, f, indent=2)

In [ ]:
with open("READY/full_cleaned.json") as f:
    data = json.load(f)

fs = data.keys()

val = []
test = []
train = []
for f in fs:
    label = data[f][1]
    if abs(sum(label) * 2 - len(label)) <= 1:
        if len(val) < 1700:
            val.append(f)
        elif len(test) < 1700:
            test.append(f)
        else:
            train.append(f)
    else:
        train.append(f) 

print len(train)
print len(val)
print len(test)

all_label = 0
pos_label = 0
for fn in train:
    label = data[fn][1]
    pos_label += sum(label)
    all_label += len(label)

print "pos: {}/neg: {}".format(pos_label, all_label - pos_label)


all_label = 0
pos_label = 0
for fn in val:
    label = data[fn][1]
    pos_label += sum(label)
    all_label += len(label)

print "pos: {}/neg: {}".format(pos_label, all_label - pos_label)


all_label = 0
pos_label = 0
for fn in test:
    label = data[fn][1]
    pos_label += sum(label)
    all_label += len(label)

print "pos: {}/neg: {}".format(pos_label, all_label - pos_label)

In [ ]:
with open('data/train_id.json', 'w') as f:
    json.dump(train, f, indent=2)
with open('data/val_id.json', 'w') as f:
    json.dump(val, f, indent=2)
with open('data/test_id.json', 'w') as f:
    json.dump(test, f, indent=2)

In [ ]:
sent='four of the cultural interest fraternities and sororities were fraternities while three are a sorority'
postprocess(sent, [], [])

In [ ]:
import nltk
get_lemmatize("crying isn't the best way", True)
#lemmatizer.lemmatize("cryin'", pos="v")

In [ ]:
import nltk
get_lemmatize("crying isn't the best way", True)

In [ ]:
import json
with open('READY/r1_training_all.json') as f:
    data = json.load(f)

words = 0
entities = 0
for k in data:
    for sent in data[k][0]:
        words += len(sent.split(' '))
        entities += sent.count('#') // 2
print 'words:', words

with open('READY/r1_training_cleaned.json') as f:
    data = json.load(f)
entities = 0
for k in data:
    for sent in data[k][0]:
        entities += sent.count('#') // 2
print 'linked:', entities
with open('READY/r2_training_all.json') as f:
    data = json.load(f)

words = 0
for k in data:
    for sent in data[k][0]:
        words += len(sent.split(' '))
print 'words:', words

with open('READY/r2_training_cleaned.json') as f:
    data = json.load(f)
entities = 0
for k in data:
    for sent in data[k][0]:
        entities += sent.count('#') // 2
print 'linked:', entities

In [ ]:
def charposition(string, char):
    pos = [] #list to store positions for each 'char' in 'string'
    for n in range(len(string)):
        if string[n] == '#':
            pos.append(n)
    return pos

import json
with open('READY/full_cleaned.json') as f:
    data = json.load(f)

In [ ]:
import pandas
import numpy

t = pandas.read_csv('data/all_csv/2-18847467-2.html.csv', '#')

t.dtypes[0] in [numpy.dtype('int64'), numpy.dtype('int32'), numpy.dtype('float32'), numpy.dtype('float64')]

print t.columns[1]

t.at[1, t.columns[1]]

In [ ]:
import multiprocessing
from itertools import product
from pathos.multiprocessing import ProcessingPool as Pool

def f(x, y, z):
    return x * y * z

cores = multiprocessing.cpu_count()
#with multiprocessing.Pool(processes=cores) as pool:
#    results = pool.starmap(merge_names, )
res = Pool().map(f, range(5), range(5), range(5))
    
# method 1: map
print res

In [ ]:
import pandas as pd

#create sample data
data = {'model': ['Lisa', 'Lisa 2', 'Macintosh 128K', 'Macintosh 512K'],
        'launched': [1983,1984,1984,1984],
        'discontinued': [1986, 1985, 1984, 1986]}

df = pd.DataFrame(data, columns = ['model', 'launched', 'discontinued'])
df
df[df['discontinued'] == 1986]

In [ ]:
import os
import json

fail = 0
success = 0
for fn in os.listdir('data/all_program/'):
    if f.endswith('json'):
        with open('data/all_program/' + fn) as f:
            print f
            data = json.load(f)
            if len(data[-1]) == 0:
                fail += 1
            else:
                success += 1

print "{}/{}".format(sucess, fail)

In [18]:
import json

with open('READY/r1_training_all.json') as f:
    r1_data = json.load(f)

with open('READY/r1_training_all.json') as f:
    r_data = json.load(f)

r_data.update(r1_data)

In [27]:
line = 0
word_counter = Counter()
for k, v in r_data.iteritems():
    for r in v[0]:
        text = r.split(' ')
        word_counter.update(text)
        line += 1

new = ['be', 'she', 'he', 'her', 'his', 'their', 'the', 'it', ',', '.', '-', 'also', 'will', 'would', 'this', 'that',
         'these', 'those', 'well', 'with', 'on', 'at', 'and', 'as', 'for', 'from', 'in', 'its', 'of', 'to', 'a',
         'an', 'where', 'when', 'by', 'not', "'s", "'nt", "make", 'who', 'have', 'within', 'without', 'what',
         'during', 'than', 'then', 'if', 'when', 'while', 'time', 'appear', 'attend', 'every', 'one', 'two', 'over',
         'both', 'above', 'only', ",", ".", "(", ")", "&", ":"]
#word_counter.update(new)
with open('/tmp/stop_words.json', 'w') as f:
    json.dump([_[0] for _ in word_counter.most_common(100)] + new, f, indent=2)
#print len(word_counter)




In [20]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import csv
import re
import random
import pandas

num = 10
num_field = 5

with open('READY/r2_training_all.json') as f:
    data = json.load(f)

with open('data/test_id.json') as f:    
    test_ids = json.load(f)
    
with open('data/table_to_page.json', 'r') as f:
    mapping = json.load(f)
    
with open('agreement_file.csv', mode='w') as fw:
    fields = []
    for i in range(1, num + 1):
        fields.extend(['wiki{}'.format(i), 'url{}'.format(i), 's{}'.format(i), 'topic{}'.format(i), 'label{}'.format(i)])
        
    csvwriter = csv.DictWriter(fw, fieldnames=fields)
    csvwriter.writeheader()
    
    small_test_ids = []
    index = 0
    buffers = {}
    for k, entry in data.iteritems():
        if k in test_ids:
            small_test_ids.append(k)
            idxs = range(len(entry[0]))
            random.shuffle(idxs)
            for idx in idxs:
                r = entry[0][idx]
                l = entry[1][idx]
                r = re.sub(r';[^#]+#', '', r)
                r = re.sub(r'#', '', r)
                print r
                if len(buffers) // num_field == num:
                    csvwriter.writerow(buffers)
                    index += 1
                    buffers = {}
                cur = len(buffers) // num_field
                buffers["url{}".format(cur + 1)] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
                buffers["s{}".format(cur + 1)] = r
                buffers["label{}".format(cur + 1)] = "Entailed" if l == 1 else "Refuted"
                if k in mapping:
                    buffers["wiki{}".format(cur + 1)] = mapping[k][1]
                    buffers['topic{}'.format(cur + 1)] = mapping[k][0]
                else:
                    buffers['wiki{}'.format(cur + 1)] = "#"
                    buffers['topic{}'.format(cur + 1)] = "None"
            
            if index > 100:
                break
    #with open('data/small_test_id.json', 'w') as f:
    #    json.dump(small_test_ids, f, indent=2)
    

In [21]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import csv
import re
import random
import pandas

num = 10
num_field = 5

with open('READY/r1_training_all.json') as f:
    data = json.load(f)

with open('data/test_id.json') as f:    
    test_ids = json.load(f)
    
with open('data/table_to_page.json', 'r') as f:
    mapping = json.load(f)
    
with open('agreement_file.csv', mode='w') as fw:
    fields = []
    for i in range(1, num + 1):
        fields.extend(['wiki{}'.format(i), 'url{}'.format(i), 's{}'.format(i), 'topic{}'.format(i), 'label{}'.format(i)])
        
    csvwriter = csv.DictWriter(fw, fieldnames=fields)
    csvwriter.writeheader()
    
    index = 0
    buffers = {}
    for k, entry in data.iteritems():
        if k in test_ids:
            small_test_ids.append(k)
            idxs = range(len(entry[0]))
            random.shuffle(idxs)
            for idx in idxs:
                r = entry[0][idx]
                l = entry[1][idx]
                r = re.sub(r';[^#]+#', '', r)
                r = re.sub(r'#', '', r)
                print r
                if len(buffers) // num_field == num:
                    csvwriter.writerow(buffers)
                    index += 1
                    buffers = {}
                cur = len(buffers) // num_field
                buffers["url{}".format(cur + 1)] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
                buffers["s{}".format(cur + 1)] = r
                buffers["label{}".format(cur + 1)] = "Entailed" if l == 1 else "Refuted"
                if k in mapping:
                    buffers["wiki{}".format(cur + 1)] = mapping[k][1]
                    buffers['topic{}'.format(cur + 1)] = mapping[k][0]
                else:
                    buffers['wiki{}'.format(cur + 1)] = "#"
                    buffers['topic{}'.format(cur + 1)] = "None"
            
            if index > 100:
                break
    with open('data/small_test_id.json', 'w') as f:
        json.dump(small_test_ids, f, indent=2)

In [5]:
from collections import Counter
import pandas

data = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v2/agreement.csv')

data = data[data.AssignmentStatus=="Approved"]

succ, fail = 0, 0
fields = 10

sent = {}
gt = {}

tp, tn, fp, fn = 0, 0, 0, 0
for i, row in data.iterrows():
    for j in range(1, fields + 1):
        """
        if row['Input.label{}'.format(j)] == "Entailed" and row['Answer.A{}'.format(j)] == "Entailed":
            tp += 1
        elif row['Input.label{}'.format(j)] == "Entailed" and row['Answer.A{}'.format(j)] == "Refuted":
            fn += 1
        elif row['Input.label{}'.format(j)] == "Refuted" and row['Answer.A{}'.format(j)] == "Entailed":
            fp += 1
        else:
            tn += 1
        """
        label = row['Answer.A{}'.format(j)]
        if row['Input.s{}'.format(j)] in sent:
            sent[row['Input.s{}'.format(j)]].append(label)
        else:
            sent[row['Input.s{}'.format(j)]] = [label]

        label = row['Input.label{}'.format(j)]
        gt[row['Input.s{}'.format(j)]] = label


for s in sent:
    stat = Counter(sent[s])
    if stat['Entailed'] > stat['Refuted'] and gt[s] == "Entailed":
        tp += 1
    elif stat['Entailed'] > stat['Refuted'] and gt[s] == "Refuted":
        fn += 1
    elif stat['Entailed'] < stat['Refuted'] and gt[s] == "Refuted":
        tn += 1
    else:
        fp += 1
    
#print "original correct: {}, original false: {}".format(orig_corr, orig_fals)
#print "pred correct: {}, pred false: {}".format(pred_corr, pred_fals)
print "TP: {}, FP: {}, FN: {}, TN: {}".format(tp, fp, fn, tn)
print "Accuracy: {}".format((tp + tn) / (tp + fp + fn + tn + 0.))
prec = tp / (tp + fp + 0.)
recall = tp / (tp + fn + 0.)
F1 = 2 * prec * recall / (prec + recall)
print "Precision: {} Recall: {} F1: {}".format(prec, recall, F1)

#for s in sent:
#    if len(sent[s]) > 1:
#        print sent[s]
p_c = []
N_ent = 0
N_ref = 0
for _, v in sent.iteritems():
    counter = Counter(v)
    total = counter['Entailed'] + counter['Refuted']
    tmp = counter['Entailed'] ** 2 + counter['Refuted'] ** 2 - total
    tmp = tmp / (total * (total - 1) + 0.0)
    p_c.append(tmp)
    N_ent += counter['Entailed']
    N_ref += counter['Refuted']

p_c = sum(p_c) / len(p_c)

p_ent = N_ent / (N_ent + N_ref + 0.)
p_ref = N_ref / (N_ent + N_ref + 0.)

p_e = p_ent ** 2 + p_ref ** 2

print p_c, p_e

kappa = (p_c - p_e) / (1 - p_e)
print "kappa = {}".format(kappa)

TP: 457, FP: 51, FN: 31, TN: 468
Accuracy: 0.91857000993
Precision: 0.899606299213 Recall: 0.936475409836 F1: 0.917670682731
0.875670307845 0.500256828808
kappa = 0.751212824262


In [6]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import json
import csv
import random
import os

with open('data/table_to_page.json', 'r') as f:
    tiny_mapping = json.load(f)
    
with open('data/simple_ids.json', 'r') as f:
    unseen = json.load(f)

random.shuffle(unseen)

count = 0
with open('blind_input.csv', 'w') as f:
    fields = ["url1", "wiki1", "topic1"]
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()

    for k in unseen:
        if os.path.exists('data/all_csv/{}'.format(k)):
            if k in tiny_mapping:
                writer.writerow({"url1": 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k, 
                                "wiki1": tiny_mapping[k][1], "topic1": tiny_mapping[k][0]})
                count += 1

            if count == 980:
                break